In [ ]:
!pip install selenium
!apt install chromium-chromedriver
!pip install requests-html

In [ ]:
import requests
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
categories = [
    "https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/",
    "https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/",
    "https://www.amazon.com/best-sellers-video-games/zgbs/videogames/",
    "https://www.amazon.com/best-sellers-books/zgbs/books/"
]

In [ ]:
categories

['https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/',
 'https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/',
 'https://www.amazon.com/best-sellers-video-games/zgbs/videogames/',
 'https://www.amazon.com/best-sellers-books/zgbs/books/']

In [ ]:
first_url = categories[0]
print(first_url)

https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/


In [ ]:
driver.get(first_url)

In [ ]:
body_el = driver.find_element_by_css_selector("body")
html_str = body_el.get_attribute("innerHTML")

In [ ]:
html_obj = HTML(html=html_str)

In [ ]:
# https://www.amazon.com/LEGO-Classic-Medium-Creative-Brick/dp/B00NHQFA1I/
# https://www.amazon.com/Crayola-Washable-Watercolors-8-ea/dp/B000HHKAE2/

# <base-url>/<slug>/dp/<product_id>/

new_links = [x for x in html_obj.links if x.startswith("/")]
new_links = [x for x in new_links if "product-reviews/" not in x]
new_links = [x for x in new_links if "/dp/" in x]
new_links = [x.split("ref", 1)[0] for x in new_links]

In [ ]:
page_links = [f"https://www.amazon.com{x}" for x in new_links]

In [ ]:
print(type(page_links))

<class 'list'>


In [ ]:
page_links

['https://www.amazon.com/Bubble-Sensory-Special-Reliever-Squeeze/dp/B08LB3HB31/',
 'https://www.amazon.com/Crayola-Ultra-Clean-Washable-Non-Toxic-Classrooms/dp/B013RQPB5C/',
 'https://www.amazon.com/Buffalo-Games-Darrell-Signature-Collection/dp/B084GF5914/',
 'https://www.amazon.com/Play-Doh-Modeling-Compound-Non-Toxic-Exclusive/dp/B00JM5GW10/',
 'https://www.amazon.com/Cards-Against-Humanity-LLC-CAHUS/dp/B004S8F7QM/',
 'https://www.amazon.com/Funko-Pop-Customizable-Exclusive-Collectible/dp/B08B77Q738/',
 'https://www.amazon.com/TeeTurtle-Unstable-Unicorns-Adventure-Expansion/dp/B085JSJPMF/',
 'https://www.amazon.com/Play-Doh-Party-Bag-Dough-Count/dp/B0037710BG/',
 'https://www.amazon.com/ThinkFun-Gravity-Marble-Logic-Girls/dp/B00IUAAK2A/',
 'https://www.amazon.com/Amerous-Inches-Magnetic-Wooden-Chess/dp/B07N5ZS5QP/',
 'https://www.amazon.com/Crayola-Light-Up-Tracing-Coloring-Amazon/dp/B07P6RFKRL/',
 'https://www.amazon.com/Just-Smarty-Interactive-Educational-Kindergarten/dp/B074PYYM51

In [ ]:
def scrape_product_page(url, title_lookup = "#productTitle", price_lookup = "#priceblock_ourprice"):
    driver.get(url)
    body_el = driver.find_element_by_css_selector("body")
    html_str = body_el.get_attribute("innerHTML")
    #print(html_str)
    html_obj = HTML(html=html_str)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    #print(product_title, product_price)
    return product_title, product_price

In [ ]:
first_product_link = page_links[0]
first_product_link

'https://www.amazon.com/Bubble-Sensory-Special-Reliever-Squeeze/dp/B08LB3HB31/'

In [ ]:
#from functools import partial 
#from tqdm import tqdm
#bar = partial(tqdm, position=0, leave=True)
#for link in bar(page_links):
i = 0
for link in page_links:
    title, price = (None, None)
    try:
        title, price = scrape_product_page(link)
    except:
        pass
    if title != None and price != None:
        i += 1
        print(str(i)+") "+str(title))
        print("Price: " + str(price))
        print("Link: " + str(link)+"\n")

1) Crayola Ultra Clean Washable Markers, Kids Indoor Activities At Home, Broad Line, 40 Classic Colors
Price: $12.22
Link: https://www.amazon.com/Crayola-Ultra-Clean-Washable-Non-Toxic-Classrooms/dp/B013RQPB5C/

2) Play-Doh Modeling Compound 10 Pack Case of Colors, Non-Toxic, Assorted Colors, 2 Oz Cans, Ages 2 & Up, (Amazon Exclusive), Multicolor
Price: $8.19
Link: https://www.amazon.com/Play-Doh-Modeling-Compound-Non-Toxic-Exclusive/dp/B00JM5GW10/

3) Cards Against Humanity
Price: $25.00
Link: https://www.amazon.com/Cards-Against-Humanity-LLC-CAHUS/dp/B004S8F7QM/

4) Funko Pop! TV: The Office - Customizable Chrome Dundie Award, Amazon Exclusive Collectible Vinyl Figure (52077)
Price: $12.99
Link: https://www.amazon.com/Funko-Pop-Customizable-Exclusive-Collectible/dp/B08B77Q738/

5) TeeTurtle Unstable Unicorns Adventures Expansion Pack - Designed to be Added to Your Unstable Unicorns Card Game (TEE4893UUEXP1)
Price: $15.00
Link: https://www.amazon.com/TeeTurtle-Unstable-Unicorns-Advent